# Knapsack problem

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

from src import qubo, data, ising, brute_force, cplex, qaoa_qmatcha
import numpy as np
from time import time
import matplotlib.pyplot as plt

## Select the problem

In [22]:
kp_type = 'small'
problems = data.load(kp_type, path='../kp_instances')
problem = problems[9]
problem

{'C': 18,
 'n': 8,
 'weights': [10, 5, 6, 5, 1, 1, 4, 4],
 'profits': [11, 7, 6, 5, 2, 4, 4, 1]}

# Classical solvers

## Brute force

In [23]:
bf_sol = None
if kp_type == 'large':
    print('I CANNOT DO THAT SIR!')
else:
    max_ram = 40E9
    st = time()
    bf_sol = brute_force.kp_brute_force(problem['profits'], problem['weights'], problem['C'], max_ram)
    et = time()
    bf_time = et - st
bf_sol

Number of threads: 15
Max samples: 31250000
Number of steps: 1


100%|██████████| 1/1 [00:00<00:00, 55.55it/s]


{'profit': array([24, 24]),
 'cost': array([18, 17]),
 'combo': [array([1, 2, 3, 4, 5]), array([0, 1, 4, 5])]}

In [24]:
# get the solution which also minimises the cost
best_indices = np.flatnonzero(bf_sol['cost'] == bf_sol['cost'].min())

print('Best solution(s) found by brute force:')
print('Profit:', bf_sol['profit'][best_indices[0]])
print('Cost:', bf_sol['cost'][best_indices[0]])
print('Combo(s):', [bf_sol['combo'][i] for i in best_indices])

print('============')
print('  RUN INFO  ')
print('============')
print(f'Run time: {bf_time :.4f} s')


Best solution(s) found by brute force:
Profit: 24
Cost: 17
Combo(s): [array([0, 1, 4, 5])]
  RUN INFO  
Run time: 0.0214 s


## CPlex

In [25]:
cplex_sol = cplex.cplex_kp_solver(problem['profits'], problem['weights'], problem['C'], problem['n'])
print('Solution found by CPLEX:')
print('Profit:', cplex_sol['profit'])
print('Cost:', cplex_sol['cost'])
print('Combo:', np.flatnonzero(cplex_sol['combo']))

print('===========================')
print('         RUN INFO          ')
print('===========================')
cplex_time = cplex_sol['runtime']
print(f'Run time: {cplex_time:.4f} s')

Knapsack problem instance as a BLIP
Problem name: KP_0008_00018

Maximize
  11*x_0 + 7*x_1 + 6*x_2 + 5*x_3 + 2*x_4 + 4*x_5 + 4*x_6 + x_7

Subject to
  Linear constraints (1)
    10*x_0 + 5*x_1 + 6*x_2 + 5*x_3 + x_4 + x_5 + 4*x_6 + 4*x_7
    <= 18  'The total weight of the knapsack must not exceed 18'

  Binary variables (8)
    x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7




Solution found by CPLEX:
Profit: 24.0
Cost: 17
Combo: [0 1 4 5]
         RUN INFO          
Run time: 0.0171 s


# Quantum solvers

## Exact diagonalization

In [18]:
# get the qubo matrix
LAMBDA = max(problem['profits'])*1.1
st = time()
Q = qubo.get_Q(problem['weights'], problem['profits'], problem['C'], LAMBDA)
t1 = time() - st

# embed the qubo matrix into an hamiltonian
st = time()
h, J, offset = ising.qubo_to_hamiltonian(Q)

'''
offset, h, J_arr = qubo_to_ising_couplings(Q).values()

J_arr = {(i, k) : J_arr[i, k] *5 for i in range(Q.shape[0]) for k in range(i+1, Q.shape[0])}
J = J_arr.copy()

for (i, k), v in J_arr.items():
    J[k, i] = v
'''
t2 = time() - st

if kp_type != 'small':
    print('I CANNOT DO THAT SIR!')
else:
    # get the diagonal of the hamiltonian
    st = time()
    diag = ising.construct_quantum_hamiltonian_diag(h, J, 0)
    t3 = time() - st
    st = time()
    diag_sol = np.argmin(diag)
    diag_sol_bits = (~brute_force.toBitstrings(diag_sol, Q.shape[0])).astype(int)
    t4 = time() - st
    cost = diag_sol_bits[:problem['n']].dot(problem['weights'])
    profit = diag_sol_bits[:problem['n']].dot(problem['profits'])
    ed_time = t1 + t2 + t3 + t4
    print('Solution found by diagonalising the Hamiltonian:')
    print('Profit:', profit)
    print('Cost:', cost, f" (but cost was: {problem['C']}" if cost > problem['C'] else '')
    print('Combo:', np.flatnonzero(diag_sol_bits[:problem['n']]))
    print('============')
    print('  RUN INFO  ')
    print('============')
    print(f'Time for QUBO: {t1:.4f} s')
    print(f'Time for Hamiltonian coeffs: {t2:.4f} s')
    print(f'Time for Hamiltonian: {t3:.4f} s')
    print(f'Time for diagonalisation: {t4:.4f} s')
    print(f'Total time: {ed_time:.4f} s')

Solution found by diagonalising the Hamiltonian:
Profit: 11
Cost: 10 
Combo: [3 4 5]
  RUN INFO  
Time for QUBO: 0.0002 s
Time for Hamiltonian coeffs: 0.0005 s
Time for Hamiltonian: 0.0602 s
Time for diagonalisation: 0.0002 s
Total time: 0.0611 s


In [19]:
offset

1372.8500000000001

In [20]:
diag[diag_sol]

-1383.8505

## QAOA in Qiskit

In [44]:
from qiskit.result import QuasiDistribution

from qiskit_algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA

from qiskit_algorithms.utils import algorithm_globals
from qiskit.circuit.library import QAOAAnsatz
from qiskit.primitives import Sampler
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

def calc_cost_and_constraint(profits, weights, solution, C):
    assert len(profits) == len(weights)
    s = len(profits)

    total_cost = sum(profits[i] * solution[i] for i in range(s))
    print("Optimum value of profit function from ED = ",total_cost)

    slack = sum((2**i) * solution[s + i] for i in range(len(solution) - (s + 1)))
    slack += (C + 1 - 2**(len(solution) - s - 1)) * solution[-1]
    print("Total value of slack variable = ", slack)

    total_weight = sum(weights[k] * solution[k] for k in range(s))
    if (total_weight <= C):
       print("Total weight less than given constraint")
    else:
       print("Weight constraint not satisfied; wrong solution")

In [45]:

# get the qubo matrix
LAMBDA = max(problem['profits']) + 2
Q = qubo.get_Q(problem['weights'], problem['profits'], problem['C'], LAMBDA)

# embed the qubo matrix into an hamiltonian
h, J, offset = ising.qubo_to_hamiltonian(Q)

st = time()
H = ising.construct_quantum_hamiltonian_qiskit(h, J, offset)
t1 = time() - st

In [13]:
result = None

if kp_type == 'large':
    print('I CANNOT DO THAT SIR!')
else:
    sampler = Sampler()
    algorithm_globals.random_seed = 10598

    optimizer = COBYLA()
    qaoa = QAOA(sampler, optimizer, reps=2)

    result = qaoa.compute_minimum_eigenvalue(H)

In [14]:
# get the solution
qaoa_qk_sol = result.best_measurement['bitstring']
qaoa_qk_sol_bits = np.logical_not(np.array([int(bit) for bit in qaoa_qk_sol])).astype(int)
print('Solution found by QAOA:')
print('Profit:', qaoa_qk_sol_bits[:problem['n']].dot(problem['profits']))
print('Cost:', qaoa_qk_sol_bits[:problem['n']].dot(problem['weights']))
print('Combo:', np.flatnonzero(qaoa_qk_sol_bits[:problem['n']]))
print('Slack:', qaoa_qk_sol_bits[problem['n']:])
calc_cost_and_constraint(problem['profits'], problem['weights'], qaoa_qk_sol_bits, problem['C'])
print('============')
print('  RUN INFO  ')
print('============')
qubo_qk_time = result.optimizer_time + t1
print(f'Time for Hamiltonian: {t1:.4f} s')
print(f'Time for QAOA: {result.optimizer_time:.4f} s')
print(f'Total time: {qubo_qk_time:.4f} s')

Solution found by QAOA:
Profit: 22
Cost: 21
Combo: [3 4 7]
Slack: [0 1 1 1 1]
Optimum value of profit function from ED =  22
Total value of slack variable =  21
Total weight less than given constraint
  RUN INFO  
Time for Hamiltonian: 0.0028 s
Time for QAOA: 46.2766 s
Total time: 46.2795 s


## QAOA in Matcha Tea

In [33]:
qaoa_qmatcha.run_qaoa_matcha(H, h, J)

KeyError: 12

# Trash

In [19]:
print(result.optimal_circuit.decompose().decompose())

global phase: -16137.0*γ[0] - 16137.0*γ[1] - 16137.0*γ[2] - 16137.0*γ[3]
         ┌─────────────┐┌──────────────────┐                  »
    q_0: ┤ U3(π/2,0,π) ├┤ Rz(-5985.0*γ[0]) ├──────────────────»
         ├─────────────┤├──────────────────┤                  »
    q_1: ┤ U3(π/2,0,π) ├┤ Rz(-6840.0*γ[0]) ├──────────────────»
         ├─────────────┤├──────────────────┤                  »
    q_2: ┤ U3(π/2,0,π) ├┤ Rz(-3420.0*γ[0]) ├──────────────────»
         ├─────────────┤├──────────────────┤                  »
    q_3: ┤ U3(π/2,0,π) ├┤ Rz(-1710.0*γ[0]) ├──────────────────»
         ├─────────────┤├─────────────────┬┘                  »
    q_4: ┤ U3(π/2,0,π) ├┤ Rz(-855.0*γ[0]) ├───────────────────»
         ├─────────────┤├─────────────────┤                   »
    q_5: ┤ U3(π/2,0,π) ├┤ Rz(4271.0*γ[0]) ├───────────────────»
         ├─────────────┤├─────────────────┤                   »
    q_6: ┤ U3(π/2,0,π) ├┤ Rz(3419.0*γ[0]) ├───────────────────»
         ├─────────────┤├──────

In [46]:
def qaoa_layer(qc: QuantumCircuit, beta: float, gamma: float, h, J):
    n = qc.num_qubits
    for i in range(n):
        qc.rx(2*beta, i)

    for i in range(n):
        qc.rz(2*gamma*h[i], i)
    
    for (i, j), v in J.items():
        qc.rzz(2*gamma*v, i, j)

    return qc

In [47]:
qc = QuantumCircuit(Q.shape[0])
qc.h(range(Q.shape[0]))
print(qc)

      ┌───┐
 q_0: ┤ H ├
      ├───┤
 q_1: ┤ H ├
      ├───┤
 q_2: ┤ H ├
      ├───┤
 q_3: ┤ H ├
      ├───┤
 q_4: ┤ H ├
      ├───┤
 q_5: ┤ H ├
      ├───┤
 q_6: ┤ H ├
      ├───┤
 q_7: ┤ H ├
      ├───┤
 q_8: ┤ H ├
      ├───┤
 q_9: ┤ H ├
      ├───┤
q_10: ┤ H ├
      ├───┤
q_11: ┤ H ├
      ├───┤
q_12: ┤ H ├
      ├───┤
q_13: ┤ H ├
      ├───┤
q_14: ┤ H ├
      ├───┤
q_15: ┤ H ├
      ├───┤
q_16: ┤ H ├
      ├───┤
q_17: ┤ H ├
      ├───┤
q_18: ┤ H ├
      ├───┤
q_19: ┤ H ├
      ├───┤
q_20: ┤ H ├
      ├───┤
q_21: ┤ H ├
      ├───┤
q_22: ┤ H ├
      ├───┤
q_23: ┤ H ├
      ├───┤
q_24: ┤ H ├
      ├───┤
q_25: ┤ H ├
      ├───┤
q_26: ┤ H ├
      ├───┤
q_27: ┤ H ├
      ├───┤
q_28: ┤ H ├
      ├───┤
q_29: ┤ H ├
      ├───┤
q_30: ┤ H ├
      ├───┤
q_31: ┤ H ├
      ├───┤
q_32: ┤ H ├
      ├───┤
q_33: ┤ H ├
      ├───┤
q_34: ┤ H ├
      ├───┤
q_35: ┤ H ├
      ├───┤
q_36: ┤ H ├
      └───┘


In [48]:
qaoa_layers = 2
betas, gammas = ParameterVector('beta', qaoa_layers), ParameterVector('gamma', qaoa_layers)
for i in range(qaoa_layers):
    qc = qaoa_layer(qc, betas[i], gammas[i], h, J)

qc.save_probabilities_dict()
#print(qc)

In [49]:
from qiskit import Aer, transpile, execute
from qiskit.primitives import Estimator

In [50]:
qc.assign_parameters({betas[0]: 0.5, gammas[0]: 0.5, betas[1]: 0.5, gammas[1]: 0.5}, inplace=True)

In [51]:
simulator = Aer.get_backend('aer_simulator')

run = execute(qc, simulator, shots=10, backend_options={'max_parallel_threads': 10})
result = run.result()
#print(result.get_counts())

In [37]:
probs = result.data()['probabilities']

probs_values = np.array(list(probs.values()))
print(list(probs.keys())[np.argmax(probs_values)])

2504


In [39]:
bitstring = brute_force.toBitstrings(list(probs.keys())[np.argmin(probs_values)], Q.shape[0])

In [40]:
bitstring

array([False,  True, False, False, False,  True,  True, False, False,
        True,  True, False])

In [82]:
estimator = Estimator(options={'shots': 1000, 'seed_simulator': 10598})

observables = [Pauli]

run = estimator.run(qc, observables=[H])

In [83]:
run.result()

MemoryError: Unable to allocate 2.00 TiB for an array with shape (137438953472,) and data type complex128

In [ ]:
result.get_counts()